In [2]:
import torch
from torch import nn
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import sklearn
from sklearn.datasets import make_circles
from sklearn.model_selection import train_test_split
import pandas as pd

#### Datasets

In [2]:
# torch.manual_seed(42)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
n_samples = 1000
X, y = make_circles(n_samples,
                    noise=0.03,
                    random_state=42)

circles = pd.DataFrame({
    "X0": X[:, 0],
    "X1": X[:, 1],
    "y": y
})
# circles.head(5)

# convert to tensor
# _X = torch.tensor(torch.from_numpy(X), dtype=torch.float16).clone().detach().requires_grad_(True)
_X = torch.from_numpy(X).type(torch.float32)
_y = torch.from_numpy(y).type(torch.int32)
# _y = torch.tensor(torch.from_numpy(y), dtype=torch.int8).clone().detach()
# _y = _y.detach().clone()

torch.manual_seed(42)
X_train, X_test, y_train, y_test = train_test_split(_X, _y, test_size=0.2, random_state=42)


In [3]:
#### Make class
class CircleModelV0(nn.Module):

    def __init__(self):
        super().__init__()
        self.layer_1 = nn.Linear(in_features=2, out_features=5)
        self.layer_2 = nn.Linear(in_features=5, out_features=1)

        # self.connected_two_layer = nn.Sequential(
        #     nn.Linear(in_features=2, out_features=5),
        #     nn.Linear(in_features=5, out_features=1)
        # )

    
    def forward(self, x):
        return self.layer_2(self.layer_1(x))
        # return self.connected_two_layer
    


In [1]:
model_0 = nn.Sequential(
    nn.Linear(in_features=2, out_features=5),
    nn.Linear(in_features=5, out_features=1)
).to(device=device)

y_preds = model_0(X_test.to(device))

_y_test = torch.unsqueeze(y_test, dim=(1)).type(torch.float16).to(device)
# y_preds.shape, _y_test.shape
# x = torch.nn.BCEWithLogitsLoss(y_preds, y_test)
pos_weight = torch.ones([len(y_preds)])
criterion = torch.nn.BCEWithLogitsLoss(pos_weight=pos_weight)
criterion = criterion.to(device)
_ypreds = torch.squeeze(y_preds, dim=0)
# _y_test = torch.tensor(_y_test, device=device, dtype=torch.float16)
# y_preds = torch.tensor(y_preds, device=device, dtype=torch.float16)
loss = criterion(y_test, _ypreds).to(device)
# _y_test.shape
# y_preds.shape

# loss


NameError: name 'nn' is not defined

In [47]:
# Instantiate model
model_0 = CircleModelV0().to(device)
with torch.inference_mode():
    untrained_preds = model_0(X_test.to(device))

print(f'Length of predictions: {len(untrained_preds)}, Shape: {untrained_preds.shape}')
print(f'\nLength of test samples: {len(X_test)}, Shape: {X_test.shape}')
print(f'\nFirst 10 predictions: {untrained_preds[: 10]}')
print(f'\nFirst 10 labels: {y_test[: 10]}')

Length of predictions: 200, Shape: torch.Size([200, 1])

Length of test samples: 200, Shape: torch.Size([200, 2])

First 10 predictions: tensor([[-0.0561],
        [-0.2016],
        [ 0.1849],
        [-0.1238],
        [-0.0378],
        [-0.1248],
        [-0.3579],
        [-0.3405],
        [ 0.1939],
        [-0.2138]], device='cuda:0')

First 10 labels: tensor([1, 0, 1, 0, 1, 1, 0, 0, 1, 0], dtype=torch.int32)


#### Setup loss function and optimizer
1. Which loss function or optimizer should we use? (problem specific)
2. For classificaiton we need binary cross entropy or categorical cross entropy

In [ ]:
10.02.17